## import

In [ ]:
pip install name_matching

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

from dateutil import parser
import datetime
from datetime import datetime
from datetime import time
import matplotlib.pyplot as plt
import unicodedata
import re
from tqdm import tqdm
from cleanco import basename
import ast

In [ ]:
from rapidfuzz import fuzz, utils
from rapidfuzz import process, fuzz

# Data

In [ ]:
# Preqin
Preqin_deals = pd.read_excel("/content/drive/MyDrive/UT_research/VC_IB/data/preqin/Preqin_deals_1983_2023.xlsx")
Preqin_deals.shape

(738455, 25)

In [ ]:
sdc_ipo_20230930 = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/sdc_ipo_20230930.csv')
sdc_ipo_20230930.drop('Unnamed: 0',axis=1,inplace=True)
sdc_ipo_20230930

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,Close_Price,initial_return,factor_2022,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,5.6875,0.137500,1.732604,8.663020,Within range,5.0,5.0,5.0,5.0,No
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,7.5000,0.071429,1.732604,13.341051,Within range,7.0,9.0,7.0,9.0,No
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,4.5000,0.000000,1.732604,9.356062,Within range,4.5,5.5,4.5,5.5,No
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,5.2500,0.050000,1.732604,13.514311,Within range,5.0,5.0,5.0,5.0,Yes
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,4.2500,-0.150000,1.732604,8.663020,Within range,5.0,6.0,5.0,6.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,16.6400,0.331200,1.000000,94.400000,Below range,16.0,18.0,16.0,18.0,No
5995,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,4.7400,0.053333,1.000000,20.100000,Within range,4.5,5.0,4.5,5.0,No
5996,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,4.7400,0.053333,1.000000,11.000000,Within range,4.5,5.0,4.5,5.0,No
5997,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,4.7100,-0.180870,1.000000,16.800000,Within range,5.0,7.0,5.0,7.0,No


In [ ]:
# drop at least one is missing
drop_list = ['PORTFOLIO COMPANY ID', 'INVESTORS / BUYERS (FIRMS)','DEAL DATE','PORTFOLIO COMPANY']
Preqin_deals = Preqin_deals.dropna(subset=drop_list)
Preqin_deals["jkl_index2"] = Preqin_deals.index
Preqin_deals

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,PORTFOLIO COMPANY STATE/ COUNTY,ENTRY REVENUE MULTIPLE,PORTFOLIO COMPANY STATUS,BOUGHT FROM / SELLERS (FIRMS),INVESTORS / BUYERS (FUNDS),LEAD PARTNERS,ACQUIRED SHARE (%),FINANCIAL ADVISORS - BUYERS,FINANCIAL ADVISORS - SELLERS,jkl_index2
0,V143231,56414,10380,Bessemer Venture Partners,Private Equity Firm (Investor),US,1983-01-01,Completed,Unspecified Round,"Maxim Integrated Products, Inc.",...,CA,NaN,Realised,NaN,NaN,NaN,NaN,NaN,NaN,0
1,V113205,216808,395,Institutional Venture Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,1
2,V113205,216808,400,InterWest Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,2
3,V113205,216808,486,Matrix Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,3
4,V113205,216808,505,Merrill Pickard Anderson & Eyre,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738450,V356543,358306,351600,UK Innovation & Science Seed Fund,Investment Company,UK,2023-09-28,Completed,Seed,Pencil Biosciences Limited,...,NaN,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,738450
738451,V356543,358306,444399,Northern Gritstone,Private Equity Firm,UK,2023-09-28,Completed,Seed,Pencil Biosciences Limited,...,NaN,NaN,Active,NaN,NaN,Duncan Johnson,NaN,NaN,NaN,738451
738452,V356550,112589,9403,Softbank China Venture Capital,Private Equity Firm (Investor),China,2023-09-28,Completed,Series D,Hydrexia Pty Ltd,...,NSW,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,738452
738453,V352746,477139,412866,SEE Fund,Other,China,2023-09-30,Completed,Unspecified Round,"Suzhou Fumailei Medical Technology Co., Ltd.",...,Jiangsu,NaN,Active,NaN,NaN,Jing He,NaN,NaN,NaN,738453


In [ ]:
preqin_sdc_merged_1024 = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/preqin_sdc_merged_1024.csv')
preqin_sdc_merged_1024.drop('Unnamed: 0',axis=1,inplace=True)
preqin_sdc_merged_1024

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Close_Price,initial_return,factor_2022,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,V345814,554599,63108,Woori Venture Partners,Private Equity Firm,South Korea,2002-06-01,Completed,Unspecified Round,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19644,V345821,554599,192019,Seoul Technology Investment,Private Equity Firm,South Korea,2012-06-01,Completed,Secondary Stock Purchase,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19645,V345826,554599,91892,Yuanta Investment,Private Equity Firm,South Korea,2017-09-25,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19646,V345829,554599,344805,G.N. Tech Venture Capital,Private Equity Firm,South Korea,2017-12-21,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No


# Matched data

In [ ]:
# hand matched lead underwriter data
hand_matching_lead  = pd.read_excel('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/investor_hand_match_1025_lead.xlsx')
hand_matching_lead.drop('Unnamed: 0',axis=1,inplace=True)
hand_matching_lead

In [ ]:
# hand matched co underwriter data
hand_matching_co  = pd.read_excel('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/investor_hand_match_1025_co.xlsx')
hand_matching_co.drop('Unnamed: 0',axis=1,inplace=True)
hand_matching_co

In [ ]:
preqin_sdc_merged_1024

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Close_Price,initial_return,factor_2022,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,V345814,554599,63108,Woori Venture Partners,Private Equity Firm,South Korea,2002-06-01,Completed,Unspecified Round,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19644,V345821,554599,192019,Seoul Technology Investment,Private Equity Firm,South Korea,2012-06-01,Completed,Secondary Stock Purchase,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19645,V345826,554599,91892,Yuanta Investment,Private Equity Firm,South Korea,2017-09-25,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19646,V345829,554599,344805,G.N. Tech Venture Capital,Private Equity Firm,South Korea,2017-12-21,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No


In [ ]:
hand_m_list_l = list(hand_matching_lead['p_s_index1'])
len(hand_m_list_l)

144

In [ ]:
hand_m_list_all = list(hand_matching_lead['p_s_index1']) +list(hand_matching_co['p_s_index1'])
len(hand_m_list_all)

184

In [ ]:
underwriter_lead_affiliated = []
for i in range (len(preqin_sdc_merged_1024)):
  if i not in hand_m_list_l:
    underwriter_lead_affiliated.append(0)
  else:
    underwriter_lead_affiliated.append(1)

preqin_sdc_merged_1024['underwriter_lead_affiliated'] = underwriter_lead_affiliated

In [ ]:
underwriter_any_affiliated = []
for i in range (len(preqin_sdc_merged_1024)):
  if i not in hand_m_list_all:
    underwriter_any_affiliated.append(0)
  else:
    underwriter_any_affiliated.append(1)

preqin_sdc_merged_1024['underwriter_any_affiliated'] = underwriter_any_affiliated

In [ ]:
# preqin_sdc_merged_1024[preqin_sdc_merged_1024['underwriter_lead_affiliated']==1].loc[:,['INVESTORS / BUYERS (FIRMS)','lead_underwriter_list','CM_CL_comanager']]

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager
30,Lehman Brothers,"['Morgan Stanley', 'Lehman Brothers']","['Credit Suisse', 'Natexis Bleichroeder Inc']"
62,Goldman Sachs Asset Management,['Goldman Sachs & Co'],"['UBS Investment Bank', 'Piper Jaffray Inc', '..."
76,JPMorgan Chase,"['Goldman Sachs & Co', 'JP Morgan', 'Barclays'...","['PNC Capital Markets', 'Raymond James & Assoc..."
181,Goldman Sachs Asset Management,['Goldman Sachs & Co'],"['Thomas Weisel Partners', 'Dain Rauscher Wess..."
565,Banc of America Securities,"['Goldman Sachs & Co', 'Credit Suisse Securiti...",['Pacific Crest Securities Inc']
...,...,...,...
19198,Credit Suisse,"['Goldman Sachs & Co', 'Merrill Lynch, Pierce,...",['Blackstone Capital Partners L']
19201,Citibank,"['Goldman Sachs & Co', 'Merrill Lynch, Pierce,...",['Blackstone Capital Partners L']
19202,Bank of America Merrill Lynch,"['Goldman Sachs & Co', 'Merrill Lynch, Pierce,...",['Blackstone Capital Partners L']
19203,Credit Suisse,"['Goldman Sachs & Co', 'Merrill Lynch, Pierce,...",['Blackstone Capital Partners L']


In [ ]:
# change the oversubscription data
over_subscription_flag =[]
for i in preqin_sdc_merged_1024['over_subscription_flag']:
  if i =='No':
    over_subscription_flag.append(0)
  else:
    over_subscription_flag.append(1)
# put it in collumns
preqin_sdc_merged_1024['over_subscription_flag_use']=over_subscription_flag

In [ ]:
preqin_sdc_merged_1024

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,underwriter_lead_affiliated,underwriter_any_affiliated,over_subscription_flag_use
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,V345814,554599,63108,Woori Venture Partners,Private Equity Firm,South Korea,2002-06-01,Completed,Unspecified Round,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19644,V345821,554599,192019,Seoul Technology Investment,Private Equity Firm,South Korea,2012-06-01,Completed,Secondary Stock Purchase,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19645,V345826,554599,91892,Yuanta Investment,Private Equity Firm,South Korea,2017-09-25,Completed,PIPE,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19646,V345829,554599,344805,G.N. Tech Venture Capital,Private Equity Firm,South Korea,2017-12-21,Completed,PIPE,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1


In [ ]:
# preqin_sdc_merged_1024.to_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/preqin_sdc_matched_1025.csv')

# results and analysis


### get and preproccess data

In [ ]:
# download the data
preqin_sdc_merged_1024 = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/preqin_sdc_matched_1025.csv')
preqin_sdc_merged_1024.drop('Unnamed: 0',axis=1,inplace=True)
preqin_sdc_merged_1024

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,underwriter_lead_affiliated,underwriter_any_affiliated,over_subscription_flag_use
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,202.751890,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,V345814,554599,63108,Woori Venture Partners,Private Equity Firm,South Korea,2002-06-01,Completed,Unspecified Round,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19644,V345821,554599,192019,Seoul Technology Investment,Private Equity Firm,South Korea,2012-06-01,Completed,Secondary Stock Purchase,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19645,V345826,554599,91892,Yuanta Investment,Private Equity Firm,South Korea,2017-09-25,Completed,PIPE,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1
19646,V345829,554599,344805,G.N. Tech Venture Capital,Private Equity Firm,South Korea,2017-12-21,Completed,PIPE,"S&D Co., Ltd.",...,10.842122,Below range,5.0,6.0,5.0,6.0,No,0,0,1


In [ ]:
# preqin deal year
preqin_sdc_merged_1024['preqin_deal_year'] = preqin_sdc_merged_1024['DEAL DATE'].str[:4]
preqin_sdc_merged_1024['preqin_deal_year'].value_counts()

In [ ]:
# use only the pre-IPO investment data. remove the post ipo investment.
preqin_sdc_merged_1025 =  preqin_sdc_merged_1024[preqin_sdc_merged_1024['DEAL DATE'] < preqin_sdc_merged_1024['issue_date']]
preqin_sdc_merged_1025.shape

(17956, 102)

In [ ]:
preqin_sdc_merged_1025['STAGE'].value_counts()

In [ ]:
# remove the add-on, pipe : nothing changes already removed
preqin_sdc_merged_1025[(preqin_sdc_merged_1025['STAGE'] != 'Add-on') | (preqin_sdc_merged_1025['STAGE'] != 'PIPE')].shape
# divide the venture debt data
preqin_sdc_merged_1025_Xvd = preqin_sdc_merged_1025[preqin_sdc_merged_1025['STAGE'] != 'Venture Debt'].copy()
preqin_sdc_merged_1025_Xvd.shape

(17430, 102)

In [ ]:
# collumns
preqin_sdc_merged_1025.columns[:50]
# preqin_sdc_merged_1025.columns[50:]

In [ ]:
# we have the underwriter affiliated data by each investor. Need to change that to by deal
preqin_sdc_merged_1025['udwr_lead_aff_by_deal'] = None
preqin_sdc_merged_1025['udwr_any_aff_by_deal'] = None
udwr_lead_aff_by_deal_list=[]
udwr_any_aff_by_deal_list=[]

In [ ]:
# change the affilation flag by investor to by deal: Lead
for jkl_i in list(preqin_sdc_merged_1025['jkl_index'].unique()):
  tempt_1 = preqin_sdc_merged_1025.groupby(['jkl_index']).get_group(jkl_i).copy()
  if tempt_1.underwriter_lead_affiliated.sum()>0:
    udwr_lead_aff_by_deal_list.append(jkl_i)

In [ ]:
# change the affilation flag by investor to by deal: Lead
for jkl_i in list(preqin_sdc_merged_1025['jkl_index'].unique()):
  tempt_1 = preqin_sdc_merged_1025.groupby(['jkl_index']).get_group(jkl_i).copy()
  if tempt_1.underwriter_any_affiliated.sum()>0:
    udwr_any_aff_by_deal_list.append(jkl_i)

In [ ]:
# get only the first row of every investment. we get the 1 row for 1 ipo
p_s_analy_df_1 = preqin_sdc_merged_1025.drop_duplicates(subset=['deal_number'], keep='first').copy()
p_s_analy_df_1

In [ ]:
# record 1 if the ipo was an lead affiliated deal
for i in range(len(p_s_analy_df_1)):
  if p_s_analy_df_1.jkl_index.iloc[i] in udwr_lead_aff_by_deal_list:
    p_s_analy_df_1.udwr_lead_aff_by_deal.iloc[i] = 1
  else:
    p_s_analy_df_1.udwr_lead_aff_by_deal.iloc[i] = 0

<ipython-input-30-a699589fa486>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_1.udwr_lead_aff_by_deal.iloc[i] = 0
<ipython-input-30-a699589fa486>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_1.udwr_lead_aff_by_deal.iloc[i] = 1


In [ ]:
# record 1 if the ipo was an co affiliated deal
for i in range(len(p_s_analy_df_1)):
  if p_s_analy_df_1.jkl_index.iloc[i] in udwr_any_aff_by_deal_list:
    p_s_analy_df_1.udwr_any_aff_by_deal.iloc[i] = 1
  else:
    p_s_analy_df_1.udwr_any_aff_by_deal.iloc[i] = 0

<ipython-input-31-4fa18031dc27>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_1.udwr_any_aff_by_deal.iloc[i] = 0
<ipython-input-31-4fa18031dc27>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_1.udwr_any_aff_by_deal.iloc[i] = 1


### create the DF for analysis

In [ ]:
p_s_analy_df_1

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,constructed_high,min_range,max_range,venture_backed,underwriter_lead_affiliated,underwriter_any_affiliated,over_subscription_flag_use,preqin_deal_year,udwr_lead_aff_by_deal,udwr_any_aff_by_deal
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,16.0,14.0,16.0,Yes,0,0,1,1997,0,0
19,V20539,25748,610,Prospect Venture Partners,Private Equity Firm,US,2003-04-01,Completed,Series A,"Jazz Pharmaceuticals, Inc.",...,NaN,NaN,NaN,No,0,0,0,2003,1,1
33,V2599,25833,537,New Enterprise Associates,Private Equity Firm (Investor),US,1997-06-01,Completed,Series A,"Financial Engines, Inc.",...,NaN,NaN,NaN,Yes,0,0,0,1997,1,1
64,V15849,26178,293,Focus Ventures,Private Equity Firm,US,2005-12-23,Completed,Series A,"Barracuda Networks, Inc.",...,NaN,NaN,NaN,No,0,0,0,2005,0,0
69,V58700,26477,1366,EdgeStone Capital Partners,Fund Manager,Canada,2004-05-06,Completed,Unspecified Round,Mitel Networks Corporation,...,NaN,NaN,NaN,No,0,0,0,2004,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19528,V305584,448167,72179,KIZOO Technology Ventures,Private Equity Firm,Germany,2019-10-09,Completed,Seed,"MAIA Biotechnology, Inc.",...,7.0,5.0,7.0,Yes,0,0,1,2019,0,0
19536,V270956,452949,1682,HBM Partners,Private Equity Firm (Investor),Switzerland,2021-11-09,Completed,Series B,"Acrivon Therapeutics, Inc.",...,18.0,16.0,18.0,No,0,0,1,2021,0,0
19556,V285199,467881,68,Atlas Venture,Private Equity Firm,US,2019-06-01,Completed,Series A,"Third Harmonic Bio, Inc.",...,18.0,16.0,18.0,Yes,0,0,1,2019,0,0
19638,V321243,520429,156832,Sixth Element Capital,Private Equity Firm,UK,2021-05-23,Completed,Series A,"Nuvectis Pharma, Inc.",...,14.0,12.0,14.0,No,0,0,1,2021,0,0


In [ ]:
p_s_analy_df_1.columns[:50]
p_s_analy_df_1.columns[50:]

In [ ]:
len(p_s_analy_df_1['jkl_index'].unique())

1295

In [ ]:
sdc_ipo_20230930.columns

Index(['jkl_index', 'deal_number', 'filing_date', 'issue_date', 'issuer',
       'main_sic_code', 'nation', 'lead_underwriter_code', 'all_managers',
       'all_mgr_role_code', 'lead_underwriter', 'cusip', 'cusip9',
       'lead_underwriter_list', 'JB_JL_jointlead', 'CM_CL_comanager',
       'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
       'selling_conces_sion_($)', 're_allow_ance_fee_($)',
       'gross_spread_in_this_mkt_($_mil)', 'principal_amount_($_mil)',
       'proceeds_amt_in_this_mkt_($_mil)', 'offer_price', 'type_of_security',
       'description', 'marketplace',
       'primary_exchange_where_issue_will_be_listed', 'HFILE', 'LFILE',
       'original_middle_of_filing_price_range', 'shares_filed_in_this_mkt',
       'amt_filed_in_this_mkt_($_mil)', 'reit_type_code', 'depos_itary',
       'unit_inv._trust', 'closed_end_fund/trust_flag_(y/n)',
       'gross_spread_as_%_of_prncpl_amt_in_this_mkt', 'AH_LFILE', 'AH_HFILE',
       'change_in_value_of_shares

In [ ]:
# change the oversubscription data
over_subscription_flag =[]
for i in sdc_ipo_20230930['over_subscription_flag']:
  if i =='No':
    over_subscription_flag.append(0)
  else:
    over_subscription_flag.append(1)
# put it in collumns
sdc_ipo_20230930['over_subscription_flag_use']=over_subscription_flag

In [ ]:
# allocate udwr_lead_aff_by_deal 0
sdc_ipo_20230930['udwr_lead_aff_by_deal'] = 0
sdc_ipo_20230930['udwr_any_aff_by_deal'] = 0

In [ ]:
for i,jkl_i in enumerate(sdc_ipo_20230930['jkl_index']):
  if jkl_i in list(p_s_analy_df_1['jkl_index']):
    sdc_ipo_20230930['udwr_lead_aff_by_deal'].iloc[i] = int(p_s_analy_df_1[p_s_analy_df_1['jkl_index'] == jkl_i].udwr_lead_aff_by_deal)
    sdc_ipo_20230930['udwr_any_aff_by_deal'].iloc[i] = int(p_s_analy_df_1[p_s_analy_df_1['jkl_index'] == jkl_i].udwr_any_aff_by_deal)

<ipython-input-38-7c53ac8df25e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_20230930['udwr_lead_aff_by_deal'].iloc[i] = int(p_s_analy_df_1[p_s_analy_df_1['jkl_index'] == jkl_i].udwr_lead_aff_by_deal)
<ipython-input-38-7c53ac8df25e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_20230930['udwr_any_aff_by_deal'].iloc[i] = int(p_s_analy_df_1[p_s_analy_df_1['jkl_index'] == jkl_i].udwr_any_aff_by_deal)


In [ ]:
# number of remaining affiliated observations
sum(sdc_ipo_20230930.drop_duplicates(subset='issuer')['udwr_any_aff_by_deal'] ==1 )

104

In [ ]:
# there are also duplicates for EU market and US market
sdc_ipo_20230930[sdc_ipo_20230930['issuer'].duplicated(keep=False)].sort_values(by=['issuer','marketplace'])
sdc_ipo_dup = sdc_ipo_20230930[sdc_ipo_20230930['issuer'].duplicated(keep=False)].copy()
sdc_ipo_dup

In [ ]:
# remove the duplicated IPO that were also in EU market
dup_eu_pub_list = list(sdc_ipo_dup[sdc_ipo_dup.marketplace == 'Euro Public'].jkl_index)
print(len(dup_eu_pub_list))
final_sdc_include_vd =  sdc_ipo_20230930[~sdc_ipo_20230930.jkl_index.isin(dup_eu_pub_list)]
final_sdc_include_vd

390


,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,over_subscription_flag_use,udwr_lead_aff_by_deal,udwr_any_aff_by_deal
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,8.663020,Within range,5.0,5.0,5.0,5.0,No,1,0,0
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,13.341051,Within range,7.0,9.0,7.0,9.0,No,1,0,0
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,9.356062,Within range,4.5,5.5,4.5,5.5,No,1,0,0
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,13.514311,Within range,5.0,5.0,5.0,5.0,Yes,1,0,0
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,8.663020,Within range,5.0,6.0,5.0,6.0,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,94.400000,Below range,16.0,18.0,16.0,18.0,No,1,0,0
5995,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,20.100000,Within range,4.5,5.0,4.5,5.0,No,1,0,0
5996,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,11.000000,Within range,4.5,5.0,4.5,5.0,No,1,0,0
5997,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,16.800000,Within range,5.0,7.0,5.0,7.0,No,1,0,0


In [ ]:
final_sdc_include_vd.to_csv('/content/drive/MyDrive/UT_research/VC_IB/results/final_sdc_df/final_sdc_include_vd.csv')

## Df for excluding Venture Debt

### including pre-ipo venture debt investment

In [ ]:
preqin_sdc_merged_1025_Xvd

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,underwriter_lead_affiliated,underwriter_any_affiliated,over_subscription_flag_use,preqin_deal_year
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1,1997
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1,1998
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1,1999
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1,1999
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,Below range,14.0,16.0,14.0,16.0,Yes,0,0,1,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19565,V285200,467881,231932,Ajax Health,Private Equity Firm (Investor),US,2022-02-16,Completed,Series B,"Third Harmonic Bio, Inc.",...,Within range,16.0,18.0,16.0,18.0,Yes,0,0,1,2022
19566,V285200,467881,326600,Commodore Capital,Fund Manager,US,2022-02-16,Completed,Series B,"Third Harmonic Bio, Inc.",...,Within range,16.0,18.0,16.0,18.0,Yes,0,0,1,2022
19567,V285200,467881,429127,Deep Track Capital,Fund Manager,US,2022-02-16,Completed,Series B,"Third Harmonic Bio, Inc.",...,Within range,16.0,18.0,16.0,18.0,Yes,0,0,1,2022
19638,V321243,520429,156832,Sixth Element Capital,Private Equity Firm,UK,2021-05-23,Completed,Series A,"Nuvectis Pharma, Inc.",...,Below range,12.0,14.0,12.0,14.0,No,0,0,1,2021


In [ ]:
# we have the underwriter affiliated data by each investor. Need to change that to by deal
preqin_sdc_merged_1025_Xvd['udwr_lead_aff_by_deal'] = None
preqin_sdc_merged_1025_Xvd['udwr_any_aff_by_deal'] = None
udwr_lead_aff_by_deal_list=[]
udwr_any_aff_by_deal_list=[]

In [ ]:
# change the affilation flag by investor to by deal: Lead
for jkl_i in list(preqin_sdc_merged_1025_Xvd['jkl_index'].unique()):
  tempt_1 = preqin_sdc_merged_1025_Xvd.groupby(['jkl_index']).get_group(jkl_i).copy()
  if tempt_1.underwriter_lead_affiliated.sum()>0:
    udwr_lead_aff_by_deal_list.append(jkl_i)

In [ ]:
# change the affilation flag by investor to by deal: Lead
for jkl_i in list(preqin_sdc_merged_1025_Xvd['jkl_index'].unique()):
  tempt_1 = preqin_sdc_merged_1025_Xvd.groupby(['jkl_index']).get_group(jkl_i).copy()
  if tempt_1.underwriter_any_affiliated.sum()>0:
    udwr_any_aff_by_deal_list.append(jkl_i)

In [ ]:
# get only the first row of every investment. we get the 1 row for 1 ipo
p_s_analy_df_2 = preqin_sdc_merged_1025_Xvd.drop_duplicates(subset=['deal_number'], keep='first').copy()
p_s_analy_df_2

In [ ]:
# record 1 if the ipo was an lead affiliated deal
for i in range(len(p_s_analy_df_2)):
  if p_s_analy_df_2.jkl_index.iloc[i] in udwr_lead_aff_by_deal_list:
    p_s_analy_df_2.udwr_lead_aff_by_deal.iloc[i] = 1
  else:
    p_s_analy_df_2.udwr_lead_aff_by_deal.iloc[i] = 0

<ipython-input-136-9a0ca4eb2fb1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_2.udwr_lead_aff_by_deal.iloc[i] = 0
<ipython-input-136-9a0ca4eb2fb1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_2.udwr_lead_aff_by_deal.iloc[i] = 1


In [ ]:
# record 1 if the ipo was an co affiliated deal
for i in range(len(p_s_analy_df_2)):
  if p_s_analy_df_2.jkl_index.iloc[i] in udwr_any_aff_by_deal_list:
    p_s_analy_df_2.udwr_any_aff_by_deal.iloc[i] = 1
  else:
    p_s_analy_df_2.udwr_any_aff_by_deal.iloc[i] = 0

<ipython-input-137-6294fe09fa70>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_2.udwr_any_aff_by_deal.iloc[i] = 0
<ipython-input-137-6294fe09fa70>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_s_analy_df_2.udwr_any_aff_by_deal.iloc[i] = 1


### exclude Venture Debt


In [ ]:
sdc_ipo_20230930_xvd = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/sdc_ipo_20230930.csv')
sdc_ipo_20230930_xvd.drop('Unnamed: 0',axis=1,inplace=True)
sdc_ipo_20230930_xvd

In [ ]:
# change the oversubscription data
over_subscription_flag =[]
for i in sdc_ipo_20230930_xvd['over_subscription_flag']:
  if i =='No':
    over_subscription_flag.append(0)
  else:
    over_subscription_flag.append(1)
# put it in collumns
sdc_ipo_20230930_xvd['over_subscription_flag_use']=over_subscription_flag

In [ ]:
# allocate udwr_lead_aff_by_deal 0
sdc_ipo_20230930_xvd['udwr_lead_aff_by_deal'] = 0
sdc_ipo_20230930_xvd['udwr_any_aff_by_deal'] = 0

In [ ]:
for i,jkl_i in enumerate(sdc_ipo_20230930_xvd['jkl_index']):
  if jkl_i in list(p_s_analy_df_2['jkl_index']):
    sdc_ipo_20230930_xvd['udwr_lead_aff_by_deal'].iloc[i] = int(p_s_analy_df_2[p_s_analy_df_2['jkl_index'] == jkl_i].udwr_lead_aff_by_deal)
    sdc_ipo_20230930_xvd['udwr_any_aff_by_deal'].iloc[i] = int(p_s_analy_df_2[p_s_analy_df_2['jkl_index'] == jkl_i].udwr_any_aff_by_deal)

<ipython-input-143-75d8fe6b6eca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_20230930_xvd['udwr_lead_aff_by_deal'].iloc[i] = int(p_s_analy_df_2[p_s_analy_df_2['jkl_index'] == jkl_i].udwr_lead_aff_by_deal)
<ipython-input-143-75d8fe6b6eca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdc_ipo_20230930_xvd['udwr_any_aff_by_deal'].iloc[i] = int(p_s_analy_df_2[p_s_analy_df_2['jkl_index'] == jkl_i].udwr_any_aff_by_deal)


In [ ]:
# number of remaining affiliated observations
sum(sdc_ipo_20230930_xvd.drop_duplicates(subset='issuer')['udwr_any_aff_by_deal'] ==1 )

98

In [ ]:
# there are also duplicates for EU market and US market
sdc_ipo_20230930_xvd[sdc_ipo_20230930_xvd['issuer'].duplicated(keep=False)].sort_values(by=['issuer','marketplace'])
sdc_ipo_dup2 = sdc_ipo_20230930_xvd[sdc_ipo_20230930_xvd['issuer'].duplicated(keep=False)].copy()
sdc_ipo_dup2

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,over_subscription_flag_use,udwr_lead_aff_by_deal,udwr_any_aff_by_deal
5,39,4.459190e+08,1994-10-28,1995-01-19,Nabisco Holdings Corp,2052,UnitedStates,GS,Goldman Sachs & Co,BM,...,1528.156715,Within range,23.0,26.0,23.0,26.0,No,1,0,0
6,44,1.097699e+09,1995-01-19,1995-01-19,Nabisco Holdings Corp,2052,UnitedStates,GS-I,Goldman Sachs International,BM,...,382.039179,NaN,NaN,NaN,NaN,NaN,No,0,0,0
20,215,4.498420e+08,1994-11-10,1995-02-02,American Standard Cos Inc,3585,UnitedStates,GS,Goldman Sachs & Co,BM,...,346.520797,Within range,19.0,22.0,19.0,22.0,No,1,0,0
21,220,1.097683e+09,1995-02-02,1995-02-02,American Standard Cos Inc,3585,UnitedStates,GS-I,Goldman Sachs International,BM,...,162.518254,NaN,NaN,NaN,NaN,NaN,No,0,0,0
30,304,4.509730e+08,1994-12-07,1995-02-09,General Magic Inc,7372,UnitedStates,GS,Goldman Sachs & Co,BM,...,100.144510,Above range,13.0,13.0,13.0,13.0,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5484,202088,3.511688e+09,2020-02-06,2020-06-03,Warner Music Group Corp,7929,UnitedStates,MS-LLC,Morgan Stanley & Co LLC,JB,...,2110.064787,NaN,NaN,NaN,NaN,NaN,No,1,0,0
5942,219194,3.860798e+09,NaN,2022-01-12,TPG Partners LLC,6799,UnitedStates,TPG-CAP-BD,TPG Capital BD LLC,BM,...,135.000000,Within range,28.0,31.0,28.0,31.0,No,1,0,0
5943,219198,3.848048e+09,2021-12-16,2022-01-12,TPG Partners LLC,6799,UnitedStates,JPM-SEC-LLC/GS,JP Morgan Securities LLC,JB,...,865.000000,Within range,28.0,31.0,28.0,31.0,No,1,0,0
5995,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,20.100000,Within range,4.5,5.0,4.5,5.0,No,1,0,0


In [ ]:
# remove the duplicated IPO that were also in EU market
dup_eu_pub_list2 = list(sdc_ipo_dup2[sdc_ipo_dup2.marketplace == 'Euro Public'].jkl_index)
print(len(dup_eu_pub_list2))
final_sdc_exclude_vd =  sdc_ipo_20230930_xvd[~sdc_ipo_20230930_xvd.jkl_index.isin(dup_eu_pub_list2)]
final_sdc_exclude_vd

In [ ]:
final_sdc_exclude_vd.to_csv('/content/drive/MyDrive/UT_research/VC_IB/results/final_sdc_df/final_sdc_exclude_vd.csv')